In [5]:
import pandas as pd
import numpy as np 
import pickle
import xgboost as xgb
import time
import graphviz
import matplotlib as plt
import re
from sklearn.decomposition import NMF

In [3]:
pik = pickle.load(open('saved_datasets/19-3-19,pipeline_A_reduced_frommaster,dateback=18/xtrain_xtest_ytrain_ytest.pickle','rb'))

In [4]:
x_train , x_test , y_train , y_test  = pik

# NMF

In [7]:
def downcast_more(df):
    float_cols = [col for col in df if (df[col].dtype=='float64' or df[col].dtype=='float32')]
    int_cols = [col for col in df if (df[col].dtype=='int64' or df[col].dtype=='int32')]
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols] = df[int_cols].astype(np.int16)
    return(df)

In [9]:
    xtrain = x_train[0]
    xtest = x_test[0]
    ndim=10
    

In [10]:
    xtrain_reduced = xtrain.filter(regex='_\d+$')
    xtest_reduced = xtest.filter(regex='_\d+$')

In [11]:
    
    reduced_cols = xtrain_reduced.columns.tolist()
    all_cols = xtrain.columns.tolist()
    other_cols = [x for x in all_cols if x not in reduced_cols]
    print('other_cols='+str(other_cols))

other_cols=['date_block_num', 'shop_id', 'item_id', 'item_category_id', 'city', 'item_type', 'item_info', 'first_sale_date_block', 'new_item', 'month', 'numdays', 'month_sales', 'mean_prevmonth_item_price', 'std_prevmonth_item_price', 'median_prevmonth_item_price', 'median_prevmonth_shop_item_price', 'prop_median_item_price']


In [13]:
    xtrain_reduced = xtrain_reduced.clip(0)
    xtest_reduced = xtest_reduced.clip(0)

In [14]:
    
    xtrain_other = xtrain[other_cols]
    xtest_other =  xtest[other_cols]
    
    xtrain_reduced = downcast_more(xtrain_reduced)
    xtest_reduced = downcast_more(xtest_reduced)

In [16]:
xtrain_other.sh

(4181736, 17)

In [17]:
    
    print(xtrain_reduced.shape)
    print(xtest_reduced.shape)
    numtrain = xtrain_reduced.shape[0]
    numtest = xtest_reduced.shape[0]

(4181736, 126)
(214200, 126)


In [ ]:
    
    concat = pd.concat([xtrain_reduced,xtest_reduced])


In [20]:
    print(concat.shape)
    nmf_obj = NMF(n_components=ndim,random_state=0,verbose=True)
    tformed = nmf_obj.fit_transform(concat)

(4395936, 126)
violation: 1.0
violation: 0.3599797066509997
violation: 0.256323923087789
violation: 0.2093658057298335
violation: 0.16684592924327332
violation: 0.1366801839801207
violation: 0.10649689868114837
violation: 0.08564007290158765
violation: 0.07337390752790145
violation: 0.06234809382582413
violation: 0.05571951657444442
violation: 0.050640622516439235
violation: 0.04638454559797909
violation: 0.042509064641931715
violation: 0.03888290853325541
violation: 0.03508710397680055
violation: 0.0307111514424653
violation: 0.02634333720238898
violation: 0.022913264297696564
violation: 0.019684408972377488
violation: 0.01720254943973163
violation: 0.015234730596262805
violation: 0.01389433749345595
violation: 0.01292396299863384
violation: 0.01225039904837192
violation: 0.011740831336456033
violation: 0.01135204629518978
violation: 0.010527332608572386
violation: 0.009887584266983338
violation: 0.009230309268147888
violation: 0.008455802374488914
violation: 0.00765918398846877
viola

In [21]:
    
    sep_bool = [True]*numtrain + [False]*numtest
    
    train_out = pd.DataFrame(tformed[sep_bool])
    test_out =  pd.DataFrame(tformed[[not x for x in sep_bool]])
    print(train_out.shape)

(4181736, 10)


In [24]:
xtrain_other.shape

(4181736, 17)

In [26]:
train_out = pd.concat([train_out,xtrain_other],axis=1)

In [28]:
train_out

,0,1,2,3,4,5,6,7,8,9,...,first_sale_date_block,new_item,month,numdays,month_sales,mean_prevmonth_item_price,std_prevmonth_item_price,median_prevmonth_item_price,median_prevmonth_shop_item_price,prop_median_item_price
0,0.000000,0.083339,9.203019,0.000000,0.039023,0.086206,0.034134,0.062925,0.662269,0.238254,...,0,0,jul,31,271433.0,149.000000,0.000000,149.000000,NaN,1.0
1,0.000000,1.098046,0.592791,0.000000,0.881526,0.709273,0.585828,0.229878,0.660228,0.586041,...,0,0,jul,31,271433.0,3270.833252,87.920151,3300.000000,NaN,1.0
2,0.000000,0.000000,0.000000,0.200511,0.510605,0.000000,0.574783,0.297556,0.000000,0.000000,...,0,0,jul,31,271433.0,600.000000,0.000000,600.000000,NaN,1.0
3,0.000000,0.010769,0.901872,0.175970,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,8,0,jul,31,271433.0,359.000000,0.000000,359.000000,NaN,1.0
4,0.000000,1.105790,0.644942,0.554015,0.870492,0.682515,0.550227,0.570035,0.563953,0.555482,...,0,0,jul,31,271433.0,600.000000,0.000000,600.000000,600.0,1.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,17,0,jul,31,271433.0,979.000000,0.000000,979.000000,NaN,1.0
6,0.000000,0.000000,0.305199,0.505928,0.000000,0.000000,0.003662,0.000000,0.614912,0.000000,...,11,0,jul,31,271433.0,249.000000,0.000000,249.000000,249.0,1.0
7,0.000000,0.000000,4.620146,2.558659,0.024995,0.000000,0.077019,0.061967,0.000000,0.000000,...,13,0,jul,31,271433.0,146.312500,15.202796,149.000000,149.0,1.0
8,0.000000,0.000000,0.000000,1.721577,0.001810,0.006179,0.014167,0.017080,0.000000,0.000665,...,15,0,jul,31,271433.0,2558.461426,143.210403,2599.000000,2599.0,1.0
9,0.000000,0.000000,0.000000,3.499046,0.004600,0.032083,0.042427,0.012469,0.002984,0.011292,...,15,0,jul,31,271433.0,2598.966553,0.147196,2599.000000,2599.0,1.0


In [ ]:
    
    train_out = pd.concat([train_out,xtrain_other],axis=1)
    test_out = pd.concat([test_out,xtest_other],axis=1)
    print(train_out.shape)

other_cols=['date_block_num', 'first_sale_date_block', 'new_item', 'numdays', 'month_sales', 'mean_prevmonth_item_price', 'std_prevmonth_item_price', 'median_prevmonth_item_price', 'median_prevmonth_shop_item_price', 'prop_median_item_price']


# other below

In [ ]:
def nmf_tform(xtrain,xtest,ndim):
    xtrain_reduced = xtrain.filter(regex='_\d+$')
    xtest_reduced = xtest.filter(regex='_\d+$')
    
    reduced_cols = xtrain_reduced.columns.tolist()
    all_cols = xtrain.columns.tolist()
    other_cols = [x for x in all_cols if x not in reduced_cols]
    print('other_cols='+str(other_cols))
#    return 5 , 5
    xtrain_reduced = xtrain_reduced.clip(0)
    xtest_reduced = xtest_reduced.clip(0)
    
    xtrain_other = xtrain[other_cols]
    xtest_other =  xtest[other_cols]
    
    xtrain_reduced = downcast_more(xtrain_reduced)
    xtest_reduced = downcast_more(xtest_reduced)
    
    print(xtrain_reduced.shape)
    print(xtest_reduced.shape)
    numtrain = xtrain_reduced.shape[0]
    numtest = xtest_reduced.shape[0]
    
    concat = pd.concat([xtrain_reduced,xtest_reduced])
    print(concat.shape)
    nmf_obj = NMF(n_components=ndim,random_state=0,verbose=True)
    tformed = nmf_obj.fit_transform(concat)
    
    sep_bool = [True]*numtrain + [False]*numtest
    
    train_out = pd.DataFrame(tformed[sep_bool])
    test_out =  pd.DataFrame(tformed[[not x for x in sep_bool]])
    print(train_out.shape)
    
    train_out = pd.concat([train_out,xtrain_other],axis=1)
    test_out = pd.concat([test_out,xtest_other],axis=1)
    print(train_out.shape)
    
    return train_out , test_out 

In [ ]:
xtrain = x_train[0]
xtrain_reduced = xtrain.filter(regex='_\d+$')

In [ ]:
#xtrain_reduced.apply(lambda x: all(x>=0),axis=0)

In [ ]:
start = time.time()
a,b = nmf_tform(x_train[0].sample(10000),x_test[0],10)
print('BREAK')
#x_train[1] , x_test[1] = nmf_tform(x_train[1],x_test[1],10)
print((time.time()-start)/60)

In [ ]:
x_train[0]

In [ ]:
dfa


In [ ]:
[not x for x in [True,False]]